### Load tensorflow

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf

In [0]:
tf.enable_eager_execution()

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Enable Eager Execution if you are using tensorflow 1.x

### Collect Data

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/drive/My Drive/GreatLakes/Lab-Residency6-Internal/prices.csv')

### Check all columns in the dataset

In [7]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns = ['date', 'symbol'], inplace = True)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
data1000 = data.head(1000)

### Convert Float64 to Float32

In [11]:
data1000.dtypes

open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

In [0]:
data1000 = data1000.astype('float32')

In [13]:
data1000.dtypes

open      float32
close     float32
low       float32
high      float32
volume    float32
dtype: object

### Divide the data into train and test sets

In [14]:
from sklearn.model_selection import train_test_split

X = data1000.drop(['close'], axis=1)
y = data1000['close']

print (X.shape)
print (X.head())




(1000, 4)
         open         low        high     volume
0  123.430000  122.309998  126.250000  2163600.0
1  125.239998  119.940002  125.540001  2386400.0
2  116.379997  114.930000  119.739998  2489500.0
3  115.480003  113.500000  117.440002  2006300.0
4  117.010002  114.089996  117.330002  1408600.0


In [15]:
print (X.shape)

(1000, 4)


### Normalize Train and Test Data 

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
transformer = Normalizer()

In [0]:
X = transformer.fit_transform (X)

In [19]:
X

array([[5.7048437e-05, 5.6530782e-05, 5.8351819e-05, 1.0000000e+00],
       [5.2480722e-05, 5.0259805e-05, 5.2606436e-05, 1.0000000e+00],
       [4.6748341e-05, 4.6165896e-05, 4.8098012e-05, 1.0000000e+00],
       ...,
       [7.6225746e-07, 7.5391358e-07, 7.7544627e-07, 1.0000000e+00],
       [6.6985353e-06, 6.6604757e-06, 6.8218496e-06, 1.0000000e+00],
       [6.4379142e-06, 6.4254232e-06, 6.6431135e-06, 1.0000000e+00]],
      dtype=float32)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)

print ('X train', X_train.shape)
print ('X test', X_test.shape)
print ('y train', y_train.shape)
print ('y test', y_test.shape)

X train (700, 4)
X test (300, 4)
y train (700,)
y test (300,)


In [21]:
X_train

array([[1.95260873e-05, 1.92729567e-05, 1.95753055e-05, 1.00000000e+00],
       [3.25567339e-06, 3.24057760e-06, 3.36386051e-06, 1.00000000e+00],
       [1.67042970e-06, 1.65513768e-06, 1.70641090e-06, 1.00000000e+00],
       ...,
       [1.04472536e-04, 1.03533064e-04, 1.04582476e-04, 1.00000000e+00],
       [1.26646424e-04, 1.23368736e-04, 1.27374806e-04, 9.99999940e-01],
       [1.16344170e-04, 1.14913542e-04, 1.18731979e-04, 9.99999940e-01]],
      dtype=float32)

In [0]:
import numpy as np

In [0]:
y_train = np.array(y_train)

## Building the graph in tensorflow

2.Define Weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

In [25]:
tf.enable_eager_execution

<function tensorflow.python.framework.ops.enable_eager_execution>

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [31]:
#Train for 100 Steps
for i in range(100):
    
    w, b = train(X_train, y_train, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, 
          loss(y_train, prediction(X_train, w, b)).numpy())

Current Loss on iteration 0 7653.4473
Current Loss on iteration 1 7333.088
Current Loss on iteration 2 7037.844
Current Loss on iteration 3 6765.7466
Current Loss on iteration 4 6514.9785
Current Loss on iteration 5 6283.8774
Current Loss on iteration 6 6070.8887
Current Loss on iteration 7 5874.6045
Current Loss on iteration 8 5693.703
Current Loss on iteration 9 5526.989
Current Loss on iteration 10 5373.3374
Current Loss on iteration 11 5231.7397
Current Loss on iteration 12 5101.2397
Current Loss on iteration 13 4980.9717
Current Loss on iteration 14 4870.1343
Current Loss on iteration 15 4767.9834
Current Loss on iteration 16 4673.8433
Current Loss on iteration 17 4587.0815
Current Loss on iteration 18 4507.124
Current Loss on iteration 19 4433.4385
Current Loss on iteration 20 4365.524
Current Loss on iteration 21 4302.9375
Current Loss on iteration 22 4245.2554
Current Loss on iteration 23 4192.0986
Current Loss on iteration 24 4143.105
Current Loss on iteration 25 4097.9565
Cur

### Get the shapes and values of W and b

In [36]:
tf.shape(w)
tf.shape(b)

<tf.Tensor: id=1967922, shape=(2,), dtype=int32, numpy=array([4, 1], dtype=int32)>

<tf.Tensor: id=1967924, shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>

In [34]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[2.6111756e-03]
 [2.5876036e-03]
 [2.6312110e-03]
 [3.2731907e+01]]
Bias:
 [32.731922]


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

### Execute the model

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
df = pd.read_csv('/content/drive/My Drive/GreatLakes/Lab-Residency6-Internal/Iris.csv')

### Splitting the data into feature set and target set

In [53]:
from sklearn.model_selection import train_test_split

X = df.drop(['Id','Species'], axis=1)
y = df['Species']

print (X.shape)
print (X.head())
print (y.shape)

(150, 4)
   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
0            5.1           3.5            1.4           0.2
1            4.9           3.0            1.4           0.2
2            4.7           3.2            1.3           0.2
3            4.6           3.1            1.5           0.2
4            5.0           3.6            1.4           0.2
(150,)


In [45]:
print (y.head())

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: Species, dtype: object


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
#Encode all categorical variables to numerical

# import library
from sklearn import preprocessing

# Create Label Encoder instance
le = preprocessing.LabelEncoder()

y = le.fit_transform(y)

In [0]:
import pandas as pd
y = pd.get_dummies(y).values

In [68]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

### Divide the dataset into Training and test (70:30)

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)

print ('X train', X_train.shape)
print ('X test', X_test.shape)
print ('y train', y_train.shape)
print ('y test', y_test.shape)

X train (105, 4)
X test (45, 4)
y train (105, 3)
y test (45, 3)


In [70]:
y_test

array([[1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0]], dtype=uint8)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
model.add(tf.keras.layers.Reshape((4,),input_shape=(4,)))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

In [0]:
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))

In [0]:
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))

In [0]:
#Add OUTPUT layer
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Fitting the model and predicting 

In [78]:
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=15)

Train on 105 samples, validate on 45 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/15
105/105 [==============================] - 1s 7ms/sample - loss: 1.3038 - acc: 0.3524 - val_loss: 1.3065 - val_acc: 0.2889
Epoch 2/15
105/105 [==============================] - 0s 158us/sample - loss: 1.2843 - acc: 0.3524 - val_loss: 1.2849 - val_acc: 0.2889
Epoch 3/15
105/105 [==============================] - 0s 149us/sample - loss: 1.2661 - acc: 0.3524 - val_loss: 1.2654 - val_acc: 0.2889
Epoch 4/15
105/105 [==============================] - 0s 143us/sample - loss: 1.2480 - acc: 0.3524 - val_loss: 1.2492 - val_acc: 0.2889
Epoch 5/15
105/105 [==============================] - 0s 183us/sample - loss: 1.2320 - acc: 0.3524 - val_loss: 1.2332 - val_acc: 0.2889
Epoch 6/15
105/105 [==============================] - 0s 187us/sample - loss: 1.2166 - acc: 0.3524 - val_loss: 1.2189 - val_acc: 0.2889
Epoch 7/15
105/105 [===========================

In [80]:
pred = model.predict(X_test)
pred

array([[0.24023114, 0.31112438, 0.44864452],
       [0.22618659, 0.29553384, 0.47827953],
       [0.22536111, 0.3013443 , 0.4732946 ],
       [0.23940948, 0.308539  , 0.45205146],
       [0.2212966 , 0.29834613, 0.4803573 ],
       [0.2227795 , 0.2967182 , 0.48050225],
       [0.2181386 , 0.29231226, 0.4895491 ],
       [0.23735894, 0.3059538 , 0.45668733],
       [0.23793314, 0.30381438, 0.4582525 ],
       [0.21576396, 0.2895507 , 0.49468535],
       [0.22157298, 0.29420197, 0.4842251 ],
       [0.23827842, 0.30797872, 0.45374283],
       [0.2162213 , 0.29166734, 0.4921114 ],
       [0.22429794, 0.2990078 , 0.47669426],
       [0.22144441, 0.29502872, 0.48352692],
       [0.23873842, 0.30208176, 0.45917985],
       [0.22361748, 0.2974639 , 0.4789186 ],
       [0.22016102, 0.2918395 , 0.48799944],
       [0.23872775, 0.30784887, 0.45342338],
       [0.2397284 , 0.30638683, 0.45388475],
       [0.22021645, 0.29306313, 0.48672035],
       [0.21927561, 0.29003826, 0.49068606],
       [0.

### Report Accuracy of the predicted values

In [84]:
model.evaluate(X_test, y_test, verbose=0)[1]

0.2888889